# 4. Flask - Forms & HTTP Methods
So far, our Mission Control has only been broadcasting signals. In this chapter, we will open two-way communication and learn to receive data and commands from our operators in the field. We will explore HTTP protocols and create standardized forms to collect key mission intelligence.

- Processing user input
- Creating forms
- `HTTP methods` **GET** & **POST** and the **request** object
- `WTF-Forms` in Flask

## 4.1. Creating a Form

- **form** `HTML form element`
  - **action** the "route" to which data should be sent after submission
  - **method** the way / method by which the data will be sent

- **label** `a tag / description for an input field`

- **input** `a field for user input`
  - **type** the type of data at the input (text, email, password, submit ..)
  - **name** the name of the input field / identifier
  - **placeholder** pre-filled text (optional)
  - **required** marks an input field that must be filled (optional)

## 4.2. Submitting a Form
- After clicking on a field of type `submit`, the data from the form is sent using the selected method (POST/GET) to the target route (URL address) from the "action".
- The `GET method` sends data in the URL - simple, insecure, with size limits, suitable for search and data display requests.
- The `POST method` sends data in the body of the **request** - secure, suitable for sensitive data or a large volume of data.

In [ ]:
<!-- stnadard HTML form -->

<form action="/login" method="POST">
    <label>Name:</label>
    <input type="text" name="username" placeholder="Username" required>
    <label>Password:</label>
    <input type="password" name="password" placeholder="Password" required>
    <input type="submit" value="OK">
</form>

## 4.3. Receiving & Using the Form
- In the target route, we `allow the given method (GET / POST)` to receive data sent from the form.
- We can then get the specific data from the form (and, for example, assign it to variables and use it).
- `request.form.get("username")` - gets data from the field **name="username"** sent via the **POST** method.
- `request.args.get("username")` - gets data from the field **name="username"** sent via the **GET** method.

In [ ]:
from flask import Flask, request
# we import the "request" object


app = Flask(__name__)


# -------------- CONFIGURATION --------------
# from a file using constants
app.config.from_pyfile("config/app_config_constants.py")


# -------------- HOMEPAGE - FORM --------------

@app.route("/form")
def form():
    return f"""
    <form action="/login" method="POST">
        <label>Your Name:</label>
        <input type="text" name="username" placeholder="Username" required>
        <label>Your Password:</label>
        <input type="password" name="password" placeholder="Password" required>
        <input type="submit" value="OK">
    </form>
    """

# -------------- LOGIN --------------

@app.route("/login", methods=["POST"]) # we allow the POST method for this route
def login():
    # getting data sent from the form via the POST method
    username = request.form.get("username")
    password = request.form.get("password")

    # getting data sent from the form via the GET method
    # username = request.args.get("username") 
    # password = request.args.get("password")
    return f"Hello, {username} you set your password to {password}!"


# -------------- STARTING THE APP --------------
if __name__ == "__main__":
    app.run()

## 4.4. WTF-Forms
Sending data over an open channel is risky and unreliable. That's why we will now deploy Flask-WTF, a robust extension that functions as an advanced encryption and validation protocol. This system will not only allow us to securely receive data but also to automatically check its integrity and correctness.

- `Flask-WTF` is an extension for creating, processing, and validating forms.
- Installation - `pip install flask-wtf`
- We will modify our application's configuration file - adding: **`SECRET_KEY = 'my_secret_something'`**
- We will create a new file for **forms** - e.g., `forms.py`
- We will pass an instance of the form into the rendered HTML page in our route.

`Application structure`:
- my_project/
  - my_app.py
  - **forms.py**
  - templates/
  - static/
  - config/

#### Configuration files in a folder, e.g., **config/**

In [ ]:
# app_config_class.py

class Config:
    """
    Set Flask config variables
    """
    # General Config
    DEBUG = True
    SECRET_KEY = 'my_secret_something' # setting the "secret key"

In [ ]:
# forms.py

from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField
from wtforms.validators import DataRequired, Length

# We create our form and add validators as needed
# We create the individual fields as variables of the "MyForm" class, which inherits from "FlaskForm"
# Multiple validators can be used at the same time

class MyForm(FlaskForm):
    username = StringField(label="New User", validators=[DataRequired()])
    email = StringField(label="Email", validators=[DataRequired()])
    password = PasswordField(label="Password", validators=[DataRequired(), 
        Length(min=6, max=20, message="Password is too short")])
    submit = SubmitField(label="OK")

In [ ]:
# my_app.py

from config.app_config_class import Config
from flask import Flask, render_template, request # we import the "request" object
from forms import MyForm # we import our form class


app = Flask(__name__)

# -------------- CONFIGURATION --------------
# from a file using a class
app.config.from_object(Config) # with "SECRET_KEY" set
"""
Setting SECRET_KEY automatically enables CSRF (Cross-Site Request Forgery) protection
= checks where (i.e., who) is sending the form with data - using the user's session

If we want to disable CSRF protection - for study/testing purposes - otherwise, we leave it on:
app.config['WTF_CSRF_ENABLED'] = False
"""


# -------------- FORM --------------
# Principle of using the form:
# using the "request" object's capabilities
@app.route("/register", methods=["GET", "POST"]) # route accepts GET and POST methods
def register():
    form = MyForm() # we create an instance of our form class 
    if request.method == "POST": # if the form is submitted via the POST method
                                 # = without validation, without checking for empty fields
        return f"{request.form.get('username')} is successfully Registered!"
    return render_template("register.html", form=form)


# form with validation and CSRF protection
# using the "WTF object's" capabilities
@app.route("/upgrade_register", methods=["GET", "POST"]) # route accepts GET and POST methods
def upgrade_register():
    form = MyForm() # we create an instance of our form class 
    if form.validate_on_submit(): # validation and check for form submission via POST method
                                  # we get data from the WTF "form" object (here "form.username.data")
        return f"{form.username.data} is successfully registered in Upgraded version!"
    return render_template("upgrade_register.html", form=form)


# -------------- STARTING THE APP --------------
if __name__ == "__main__":
    app.run()

#### HTML pages are in the **templates/** folder

In [ ]:
<!-- register.html -->

<form method = "POST" action = "{{ url_for('register') }}">
    <p>Standard register form</p>
    <p>{{ form.username.label }} {{ form.username(size=30) }}</p>
    <p>{{ form.email.label }} {{ form.email(size=30) }}</p>
    <p>{{ form.password.label }} {{ form.password(size=20) }}</p>
    <p>{{ form.submit() }}</p>
</form>

In [ ]:
<!-- upgrade_register.html -->

<form method = "POST" action = "{{ url_for('upgrade_register') }}">
    <p>Upgrade register form</p>
    {{ form.hidden_tag() }} <!-- here is where CSFR  token is placed automatically by Flask-WTF -->
    <!-- the token is used to validate the form  (validate_on_submit) -->

    <p>{{ form.username.label }} {{ form.username(size=30) }}</p>
    <p>{{ form.email.label }} {{ form.email(size=30) }}</p>
    <p>{{ form.password.label }} {{ form.password(size=20) }}</p>

    <p>{{ form.submit() }}</p>
</form>

### **Practise I**

**1. Sending a Command to a Mining Drone**

Create a simple form (using WTF-Forms) to send a text command to an autonomous drone.

1.  **Create the form** in `forms.py` named `DroneCommandForm`. It will contain:
    * `drone_id` (a `StringField`, required, for the drone's identification, e.g., "DR-7")
    * `command` (a `StringField`, required, for the command itself, e.g., "SCAN_SECTOR_GAMMA")
    * `submit` (a submit button)

2.  **Create a route** `/send_command` that will display an HTML page (e.g., `drone_command.html`) with our form and will handle both methods (`GET` and `POST`).

3.  After a successful submission (`validate_on_submit`), the form will return a simple confirmation message, for example:
    `f"Command '{form.command.data}' has been sent to drone '{form.drone_id.data}'."`

### **Practise II**

**2. Energy Shield Calibration**

Create an interface to change the power level of the station's energy shield.

**Data:**
At the beginning of your file, create a simple dictionary that represents the current shield settings.
```python
SHIELD_SETTINGS = {'power_level': 50, 'frequency': 75.5}
```

1.  **Create a form** `ShieldForm` with one field:
    * `new_power_level` (an `IntegerField`, required)
    * `submit` (a submit button)

2.  **Create a route** `/shield_calibration` (`GET` and `POST`).
    * On a `GET` request, it will display a template (e.g., `shield_calibration.html`) that shows the **current** `power_level` from the `SHIELD_SETTINGS` dictionary and also displays the form for changing it.
    * On a `POST` request (`validate_on_submit`):
        * It updates the `'power_level'` value in the `SHIELD_SETTINGS` dictionary to the value from the form.
        * It **redirects** back to the same page (`/shield_calibration`), so we can verify that the power level value has actually changed.

### **Project (Homework): Crew Member Recruitment**

**Mission:** As the mission demands grow, new specialists need to be brought on board. Our task is to create an **onboarding form** for Mission Control and connect it to our application.

`Basic application structure`:
- my_project/
  - my_app.py
  - routes.py
  - **forms.py**
  - templates/
  - static/
  - config/

1.  **New Form:**
    - Create a new file `forms.py` and in it, create a class, e.g., `NewCrewMemberForm`, which will contain fields for entering a new crew member (i.e., `pilot_name`, `specialization`, etc.).

2.  **Route for the Form:**
    - Create a new route `/onboarding` that will accept `GET` and `POST` requests and will display and process this recruitment form.

3.  **Logic Upgrade:**
    - After a successful form submission (`validate_on_submit`), take the data and **add a new member** to our crew.
    - Arrange for them to automatically get an `id` that follows on from the `id` of the last added member (if the last record has an `id` value of "4", the next record gets "5", and the next "6" ..) and for the `status` to be set to `pending`.

4.  **Navigation Upgrade:**
    - Add the `/onboarding` route to the navigation menu.

5.  **Output Check:**
    - After adding a new member, **redirect** the applicant from the `/onboarding` route to the `/crew_members` page and check the updated crew list.

---
#### © Jiří Svoboda (George Freedom)
- Web: https://GeorgeFreedom.com
- LinkedIn: https://www.linkedin.com/in/georgefreedom/
- Book me: https://cal.com/georgefreedom